# Extract the frames out of the videos and set up the classifcation framework

## Setup

In [1]:
import os
import cv2
import shutil
import random

### Setup Global Variables

In [2]:
# Declareer globale variabelen
global interval 

# Initialiseer de variabelen
interval = 3

### The table data met labels (copy of the collect data)

In [3]:
# Create an array containing the table data with an index
table_data = [
    ["Label", "Gaze Direction", "Sentence"],
    ["forward", "Forward", "Look forward."],
    ["left", "Left", "Look to the left."],
    ["right", "Right", "Look to the right."],
    ["mirror_interior", "Interior Mirror", "Look at the interior mirror."],
    ["mirror_right", "Right Side Mirror", "Look at the right side mirror."],
    ["mirror_left", "Left Side Mirror", "Look at the left side mirror."],
    ["shoulder_right", "Right Shoulder", "Look over your right shoulder."],
    ["shoulder_left", "Left Shoulder", "Look over your left shoulder."],
    ["dashboard_straight_down", "Dashboard Straight Down", "Look straight down at the dashboard."],
    ["dashboard_down_right", "Dashboard Down Towards Center Console", "Look down towards the center console."],
    ["forward_right", "Forward Right", "Look forward and to the right."],
    ["forward_left", "Forward Left", "Look forward and to the left."]
]

# To access, for example, the row for "forward_right" and its Sentence column:
# Note that Python is 0-indexed, so row 11 (forward_right) is at index 10 in the array
# print(table_data[11][2])  # Output: Look forward and to the right.


## Iterate over the collected data, call the extract function and divide the files on the directories train, test and val. 

### Interate over the labels and ranges

In [12]:
base_path = r"C:\GazeDetection\test_recording_correct_upload"  # Base path

for row in table_data[1:]:  # Start at index 1 to skip the header
    label = row[0]
    print(f"Label: {label}")
    outputpad = os.path.join(base_path, "train", label)  # Creates the full path for the new directory (train)

    if not os.path.exists(outputpad):  # Check if the directory does not exist yet
        os.makedirs(outputpad)  # Create the directory

    for i in range(1, 11):  # Iterate over the numbers 1 through 10
        filepad = f"{base_path}\\{label}\\{i}"  # Create the path for this label and number

        # Handle camera 1
        file1 = f"{filepad}\\camera_1.avi"  # Filename for camera 1
        camera = 1
        extract_frames(file1, outputpad, camera, interval, i)

        # Handle camera 2
        file2 = f"{filepad}\\camera_2.avi"  # Filename for camera 2
        camera = 2
        extract_frames(file2, outputpad, camera, interval, i)

    # Divide to test directory after finishing all iterations of i
    source_directory = outputpad  # Source folder
    destination_directory = os.path.join(base_path, "test", label)  # Target folder

    if not os.path.exists(destination_directory):  # Check if the directory does not exist yet
        os.makedirs(destination_directory)  # Create the directory

    moved_files_count = divide(source_directory, destination_directory, 10)  # Moving 10% of the files
    print(f"Moved {moved_files_count} files to directory test")

    # Divide to val directory after finishing all iterations of i
    destination_directory = os.path.join(base_path, "val", label)  # Target folder

    if not os.path.exists(destination_directory):  # Check if the directory does not exist yet
        os.makedirs(destination_directory)  # Create the directory

    moved_files_count = divide(source_directory, destination_directory, 10)  # Moving 10% of the files
    print(f"Moved {moved_files_count} files to directory val")


Label: forward
Aantal bestanden in de directory train: 1016
Moved 101 files to directory test
Aantal bestanden in de directory train: 915
Moved 91 files to directory val
Label: left
Aantal bestanden in de directory train: 920
Moved 92 files to directory test
Aantal bestanden in de directory train: 828
Moved 82 files to directory val
Label: right
Aantal bestanden in de directory train: 1018
Moved 101 files to directory test
Aantal bestanden in de directory train: 917
Moved 91 files to directory val
Label: mirror_interior
Aantal bestanden in de directory train: 1014
Moved 101 files to directory test
Aantal bestanden in de directory train: 913
Moved 91 files to directory val
Label: mirror_right
Aantal bestanden in de directory train: 1016
Moved 101 files to directory test
Aantal bestanden in de directory train: 915
Moved 91 files to directory val
Label: mirror_left
Aantal bestanden in de directory train: 1018
Moved 101 files to directory test
Aantal bestanden in de directory train: 917
Mo

### Extract de frames out the videos and save in the output_dir

In [4]:
def extract_frames(video_path, output_dir, camera, interval, frequentie):
    """
    Extract frames from a video file and save them to a specified directory.

    Parameters:
    - video_path: Path to the video file.
    - output_dir: Directory where extracted frames will be saved.
    - interval: Interval between frames to extract. Default is every 3rd frame.
    - max_frames: Maximum number of frames to extract. Default is 30 frames.

    Returns:
    - None
    """

    # Initialize video object
    cap = cv2.VideoCapture(video_path)

    # Check if the video was opened successfully
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return
    
    # Retrieve the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Initialize frame index
    frame_idx = 0

    # List for storing extracted frames (optional, depending on your needs)
    extracted_frames = []

    while True:
        # Read the current frame
        ret, frame = cap.read()

        # Stop the loop if the frame is not successfully read or if we have already extracted the desired number of frames
        if not ret or len(extracted_frames) >= total_frames:
            break

        # Check if the current frame index is a multiple of the interval
        if frame_idx % interval == 0:
            extracted_frames.append(frame)  # Optional: Store frame in list
            # Save frame as an image file
            filename = f"{label}_camera_{camera}_cyclus_{frequentie}_frame_{frame_idx}.jpg"
            frame_file = os.path.join(output_dir, filename)
            cv2.imwrite(frame_file, frame)

        frame_idx += 1

    # Release the video capture object
    cap.release()


### Moves an percentage of the files from one folder to another.

In [11]:
def divide(source_dir, target_dir, percentage):
    """
    Moves a specified percentage of photos from a source directory to a target directory randomly.

    Parameters:
    source_dir (str): The path to the source directory containing the photos.
    target_dir (str): The path to the target directory where the photos will be moved.
    percentage (int): The percentage of files to be moved.

    Returns:
    int: The number of files moved.
    """
    # Check if the target directory exists, create it if not
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # List of all files in the source directory
    files = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]
    print("Aantal bestanden in de directory train:", len(files))

    # Determine the percentage of total files to move (round down)
    sample_size = max(1, len(files) * percentage // 100)  # Ensure at least one file is moved

    # Randomly select the calculated percentage of files
    selected_files = random.sample(files, sample_size)

    # Move the selected files to the target directory
    for file in selected_files:
        shutil.move(os.path.join(source_dir, file), os.path.join(target_dir, file))

    return len(selected_files)